<a href="https://colab.research.google.com/github/FastBackendML/FastBackendML/blob/main/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.10 Использование CPU и GPU на примере простой НС

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t1 = torch.arange(1,6, dtype=torch.int32, device=device)
t2 = torch.ones(15, dtype=torch.float32, device=device).view(3,5)
t_res = t2.matmul(t1.float()).to(device='cpu')

In [ ]:
t_res

tensor([15., 15., 15.])

Task 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
targets = torch.rand(1000).to(device)
predict = torch.rand(1000).to(device)

In [ ]:
Q = torch.sum(torch.pow((targets - predict), 2)) / 1000

In [ ]:
Q = Q.to('cpu')

In [ ]:
Q

tensor(169.7121)

Task 3

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

w = torch.rand(1, 16).to(device) # вектор-строка
X = torch.rand(16, 16).to(device) # матрица 16 x 16
I = torch.eye(16).to(device) # единичная матрица 16 x 16
lm = 0.5 # значение лямбда

In [ ]:
Q = w.matmul((X.add(I.mul(lm)))).matmul(w.transpose(0,1))

In [ ]:
Q.to('cpu')

tensor([[27.8480]])

Task 4

In [ ]:
import torch
# все эти переменные в программе не менять, только тензоры разместить на device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

W = torch.rand(3) * 10 - 5
bias = torch.rand(1) * 100 - 50

batch_size = 8 # размер мини-батча
X = torch.empty(batch_size, 3).normal_(mean=-1.0, std=4.0).to(device)
predict = (X.to(device).mv(W.to(device)) + bias.to(device)).sigmoid()

# 1.11 Персептрон - возможности классификации образов

In [ ]:
import torch

# тензор X в программе не менять
batch_size = 32 # количество входных данных
X = torch.tensor(list(map(float, input().split())), dtype=torch.float32).view(batch_size, 2)

def line_coeff(x0, y0, x1, y1):
    w1 = y1 - y0
    w2 = -(x1 - x0)
    w0 = x1 * y0 - x0 * y1
    return -w0, -w1, -w2


def activate(x):
    return torch.sign(x)


w10, w11, w12 = line_coeff(-4, -6, -5, -3)
w20, w21, w22 = line_coeff(-5, -3, 1, 5)
w30, w31, w32 = line_coeff(1, 5, 10, 9)
w0, w1, w2, w3 = 2.5, 1, 1, 1


W1 = torch.tensor([(w11, w12), (w21, w22), (w31, w32)], dtype=torch.float32)
bias1 = torch.tensor([w10, w20, w30], dtype=torch.float32)
W2 = torch.tensor([w1, w2, w3], dtype=torch.float32)
bias2 = torch.tensor([w0], dtype=torch.float32)


X1_out = activate(X @ W1.permute(1, 0) + bias1)
predict = activate(X1_out @ W2 + bias2)

In [ ]:
import torch


# тензор X в программе не менять
batch_size = 32 # количество входных данных
X = torch.tensor(list(map(float, input().split())), dtype=torch.float32).view(batch_size, 2)

W1 = torch.tensor([[-1.8,  1.0],
                   [-0.75,  1.0]])
bias1 = torch.tensor([-2.4, -4.5])
# Веса выходного слоя сделаем равные (равный вклад двух нейронов скрытого слоя)
W2 = torch.tensor([1.0, 1.0])
# Если оба выхода скрытого слоя нулевые, нужно сместить их сумму ниже нуля
bias2 = torch.tensor([0.5])

# Функция активации общая для всех трёх нейронов
act = lambda z: torch.where(z > 0.0, torch.ones_like(z), -torch.ones_like(z))

# Находим выход скрытого слоя
h = act(X @ W1.t() + bias1)

# Находим выход выходного слоя
predict = act(h @ W2 + bias2)
# predict


# 2.1 Идея обучения НС градиентным алгоритмом

Task 1

In [ ]:
def func(x):
    return 0.5 * x + 0.2 * x ** 2 - 0.1 * x ** 3


def df(x):
    return 0.5 + 0.4 * x - 0.3 * x ** 2


# здесь продолжайте программу
n = 0.01
x0 = -4
N = 200
x = x0

for i in range(N):
    dif = df(x)
    x = x - n * dif

Task 2

In [ ]:
import torch

# Целевая функция
def func(x):
    return 0.1 * x**2 - torch.sin(x) + 5.

# Данные
coord_x = torch.arange(-5.0, 5.0, 0.1)
coord_y = func(coord_x).view(-1, 1)  # [N, 1]

# Подготовка признаков: [1, x, x^2, x^3]
X = torch.stack([torch.ones_like(coord_x), coord_x, coord_x**2, coord_x**3], dim=1)  # [N, 4]

# Начальные параметры
w = torch.zeros((4, 1), dtype=torch.float32)  # [4, 1]
eta = torch.tensor([0.1, 0.01, 0.001, 0.0001]).view(4, 1)  # [4, 1]
N = 200
sz = coord_x.size(0)

# Градиентный спуск
for i in range(N):
    y_pred = X @ w  # [N, 1]
    error = y_pred - coord_y  # [N, 1]
    grad = (2 / sz) * (X.T @ error)  # [4, 1]
    w = w - eta * grad  # Поэлементное обновление

# Итоговая ошибка
y_pred = X @ w
Q = torch.mean((y_pred - coord_y)**2).item()

# 2.3 Функции активации и потерь в PyTorch

Task 1

In [ ]:
import torch

# значения x, func, predict не менять
x = torch.arange(-3, 3, 0.1)
func = x ** 2 - 2 * torch.cos(x) - 5
predict = func + torch.empty_like(func).normal_(0, 0.5)

loss_func = torch.nn.MSELoss()
Q = loss_func(predict, func).float().item()

Q_mse = torch.sum((predict - func)**2) / len(predict)
Q_mse = Q_mse.float().item()

Task 2

In [ ]:
import torch

# значения predict, target не менять
batch_size = 8
target = torch.randint(0, 2, (batch_size, 1), dtype=torch.float32) # целевые значения
predict = torch.empty(batch_size, 1).normal_(0, 2.0) # прогнозные значения

loss_func = torch.nn.BCEWithLogitsLoss()
Q = loss_func(predict, target).float().item()
sig_preds = torch.sigmoid(predict)
eps = 1e-7  # защита от log(0)
Q_bce = -torch.sum(target * torch.log(sig_preds + eps) + (1 - target) * torch.log(1 - sig_preds) + eps) / batch_size
Q_bce = Q_bce.float().item()
print(Q_bce, Q)

0.9961713552474976 0.9961714744567871


# 2.4 Автоматическое дифференцирование

Task 1

In [ ]:
import torch

d = float(input()) # значение d в программе не менять
t = torch.tensor([d], requires_grad=True)
f = 3 * t**2 + 5 * t - 2
f.backward()
print(f)
print(t.data, t.grad)

50
tensor([7748.], grad_fn=<SubBackward0>)
tensor([50.]) tensor([305.])


Task 2

In [ ]:
import torch

g = float(input()) # значение g в программе не менять
d = float(input()) # значение d в программе не менять

t = torch.tensor([d], requires_grad=True)

f = -(g * t**2) / 2 + g * torch.exp(-t) + 1.5

f.backward()

print(t.data, t.grad)
print(f)

50
30
tensor([30.]) tensor([-1500.])
tensor([-22498.5000], grad_fn=<AddBackward0>)


Task 3

In [ ]:
import torch

xx = float(input()) # значение xx в программе не менять
x = torch.tensor(xx, dtype=torch.float32)

x0, x1, x2, x3 = map(float, input().split()) # переменные x0, x1, x2, x3 в программе не менять

w0 = torch.tensor(x0, dtype=torch.float32, requires_grad=True)
w1 = torch.tensor(x1, dtype=torch.float32, requires_grad=True)
w2 = torch.tensor(x2, dtype=torch.float32, requires_grad=True)
w3 = torch.tensor(x3, dtype=torch.float32, requires_grad=True)

a = w0 + w1 * x + w2 * x**2 + w3 * x**3
f = 0.5 * x**2 - 2 * x - 5
L = (a - f)**2
L.backward()

# 2.5 Оптимизаторы. Реализация SGD на PyTorch

Task 1

In [ ]:
import torch

def func(x):
    return 0.2 * (x - 2) ** 2 - 0.3 * torch.cos(4 * x)


x0 = 0.0 # начальное значение точки минимума
lr = 0.1 # шаг обучения
N = 200 # число итераций градиентного алгоритма
x = torch.tensor([x0], dtype=torch.float32, requires_grad=True)

for i in range(N):
  y = func(x)
  y.backward()
  x.data = x.data - lr * x.grad
  x.grad.zero_()


Task 2

In [ ]:
import torch
import torch.optim as optim

def func(x):
    return 0.2 * (x - 2) ** 2 - 0.3 * torch.cos(4 * x)


lr = 0.1 # шаг обучения
x0 = 0.0 # начальное значение точки минимума
N = 200 # число итераций градиентного алгоритма
x = torch.tensor([x0], requires_grad=True)
optimizer = optim.RMSprop(params=[x], lr=lr)

for _ in range(N):
  y = func(x)
  y.backward()
  optimizer.step()
  optimizer.zero_grad()

Task 3

In [ ]:
import numpy as np
import torch
import torch.optim as optim

x_train = torch.tensor([(5.8, 1.2), (5.6, 1.5), (6.5, 1.5), (6.1, 1.3), (6.4, 1.3), (7.7, 2.0), (6.0, 1.8), (5.6, 1.3), (6.0, 1.6), (5.8, 1.9), (5.7, 2.0), (6.3, 1.5), (6.2, 1.8), (7.7, 2.3), (5.8, 1.2), (6.3, 1.8), (6.0, 1.0), (6.2, 1.3), (5.7, 1.3), (6.3, 1.9), (6.7, 2.5), (5.5, 1.2), (4.9, 1.0), (6.1, 1.4), (6.0, 1.6), (7.2, 2.5), (7.3, 1.8), (6.6, 1.4), (5.6, 2.0), (5.5, 1.0), (6.4, 2.2), (5.6, 1.3), (6.6, 1.3), (6.9, 2.1), (6.8, 2.1), (5.7, 1.3), (7.0, 1.4), (6.1, 1.4), (6.1, 1.8), (6.7, 1.7), (6.0, 1.5), (6.5, 1.8), (6.4, 1.5), (6.9, 1.5), (5.6, 1.3), (6.7, 1.4), (5.8, 1.9), (6.3, 1.3), (6.7, 2.1), (6.2, 2.3), (6.3, 2.4), (6.7, 1.8), (6.4, 2.3), (6.2, 1.5), (6.1, 1.4), (7.1, 2.1), (5.7, 1.0), (6.8, 1.4), (6.8, 2.3), (5.1, 1.1), (4.9, 1.7), (5.9, 1.8), (7.4, 1.9), (6.5, 2.0), (6.7, 1.5), (6.5, 2.0), (5.8, 1.0), (6.4, 2.1), (7.6, 2.1), (5.8, 2.4), (7.7, 2.2), (6.3, 1.5), (5.0, 1.0), (6.3, 1.6), (7.7, 2.3), (6.4, 1.9), (6.5, 2.2), (5.7, 1.2), (6.9, 2.3), (5.7, 1.3), (6.1, 1.2), (5.4, 1.5), (5.2, 1.4), (6.7, 2.3), (7.9, 2.0), (5.6, 1.1), (7.2, 1.8), (5.5, 1.3), (7.2, 1.6), (6.3, 2.5), (6.3, 1.8), (6.7, 2.4), (5.0, 1.0), (6.4, 1.8), (6.9, 2.3), (5.5, 1.3), (5.5, 1.1), (5.9, 1.5), (6.0, 1.5), (5.9, 1.8)])
y_train = torch.tensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1])
y_train = y_train.float()

total = len(y_train) # размер обучающей выборки
n_features = 3 # число параметров w0, w1, w2
lr = 0.1 # шаг обучения
N = 500 # число итераций градиентного алгоритма

np.random.seed(1) # установка "зерна" генератора датчика случайных чисел

X = torch.ones(total, 3)
X[:, 1:3] = x_train # матрица формата [(1, x11, x12), (1, x21, x22), ..., (1, xn1, xn2)]

w = torch.empty(n_features).uniform_(-1e-5, 1e-5) # начальные значения вектора w
w.requires_grad_(True)

# здесь продолжайте программу
BCE_loss = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=[w], lr=lr)
for _ in range(N):
    k = np.random.randint(0, total)
    loss = BCE_loss(X[k] @ w, y_train[k])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Q = torch.mean((torch.sign(X @ w) == (y_train * 2 - 1)).float())


Task 4

In [ ]:
import numpy as np
import torch
import torch.optim as optim

def model(x, w1, w2, b1, b2):
    x = w1 @ x + b1
    x = torch.tanh(x)
    x = w2 @ x + b2
    return x


np.random.seed(1) # установка "зерна" генератора датчика случайных чисел
torch.manual_seed(123)

W1 = torch.empty(2, 2).normal_(0, 1e-5)
bias1 = torch.rand(2, requires_grad=True)
W2 = torch.empty(1, 2).normal_(0, 1e-5)
bias2 = torch.rand(1, requires_grad=True)

W1.requires_grad_(True)
W2.requires_grad_(True)

# обучающая выборка
n_items = 20
C00 = torch.empty(n_items, 2).normal_(0, 1)
C11 = torch.empty(n_items, 2).normal_(0, 1) + torch.FloatTensor([5, 5])
C01 = torch.empty(n_items, 2).normal_(0, 1) + torch.FloatTensor([0, 5])
C10 = torch.empty(n_items, 2).normal_(0, 1) + torch.FloatTensor([5, 0])

x_train = torch.cat([C00, C11, C01, C10])
y_train = torch.cat([torch.ones(n_items * 2), torch.zeros(n_items * 2)])

lr = 0.01  # шаг обучения
N = 1000  # число итераций при обучении
total = y_train.size(0) # размер обучающей выборки

# здесь продолжайте программу

Loss = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=[W1, W2, bias1, bias2], lr=lr)

for _ in range(N):
    k = np.random.randint(0, total)
    y = model(x_train[k], W1, W2, bias1, bias2)
    loss = Loss(y, y_train[k].unsqueeze(0))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

a = (np.sign([model(x, W1, W2, bias1, bias2).item() for x in x_train]) + 1) / 2
Q = np.mean(a == y_train.numpy())
Q

np.float64(0.95)

# 2.6 Классы nn.Linear и nn.Module

Task 1

In [ ]:
import torch
import torch.nn as nn

# тензор x в программе не менять
x = torch.tensor(list(map(float, input().split())), dtype=torch.float32)

# здесь продолжайте программу
layer = nn.Linear(16, 1, bias=False)
layer.weight = nn.Parameter(torch.ones(1, 16))
round(layer(x).item(), 1)

 -1.6 7.8 3.6 0.1 3.7 5.9 -1.8 10.0 -0.6 -2.8 0.8 0.4 9.5 2.4 4.9 9.6


51.9

Task 2

In [ ]:
import torch
import torch.nn as nn

# здесь объявляйте класс модели (обязательно до тензора x)
class TwoLayerModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(3, 2)
    self.layer2 = nn.Linear(2, 1)

  def forward(self, x):
    x = self.layer1(x)
    x = torch.sigmoid(x)
    x = self.layer2(x)
    return x

model = TwoLayerModel()
model.eval()

x = torch.rand(3) # тензор x в программе не менять

# здесь продолжайте программу
predict = model(x)

Task 3

In [ ]:
import torch
import torch.nn as nn

# здесь объявляйте класс модели (обязательно до тензора x)
class ThreeLayersModel(nn.Module):
  def __init__(self, n_input, layer1_input, layer2_input, n_output):
    super().__init__()
    self.layer1 = nn.Linear(n_input, layer1_input)
    self.layer2 = nn.Linear(layer1_input, layer2_input)
    self.layer3 = nn.Linear(layer2_input, n_output)

  def forward(self, x):
    x = self.layer1(x)
    x = torch.tanh(x)
    x = self.layer2(x)
    x = torch.tanh(x)
    x = self.layer3(x)
    x = torch.tanh(x)
    return x


model = ThreeLayersModel(32, 10, 12, 1)
model.eval()

batch_size = 4
x = torch.rand(batch_size, 32) # тензор x в программе не менять

# здесь продолжайте программу
predict = model(x)

In [ ]:
predict

tensor([[0.3561],
        [0.2227],
        [0.3309],
        [0.3334]], grad_fn=<TanhBackward0>)

Task 4

In [ ]:
import torch
import torch.nn as nn

# здесь объявляйте класс модели (обязательно до тензора x_train)
class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(2, 3)
    self.layer2 = nn.Linear(3, 1)

  def forward(self, x):
    x = self.layer1(x)
    x = torch.tanh(x)
    x = self.layer2(x)
    x = torch.sigmoid(x)
    return x

model = MyModel()
model.eval()

# тензор x_train в программе не менять
x_train = torch.tensor([(5.8, 1.2), (5.6, 1.5), (6.5, 1.5), (6.1, 1.3), (6.4, 1.3),
                        (7.7, 2.0), (6.0, 1.8), (5.6, 1.3), (6.0, 1.6), (5.8, 1.9),
                        (5.7, 2.0), (6.3, 1.5), (6.2, 1.8), (7.7, 2.3), (5.8, 1.2),
                        (6.3, 1.8), (6.0, 1.0), (6.2, 1.3), (5.7, 1.3), (6.3, 1.9),
                        (6.7, 2.5), (5.5, 1.2), (4.9, 1.0), (6.1, 1.4), (6.0, 1.6),
                        (7.2, 2.5), (7.3, 1.8), (6.6, 1.4), (5.6, 2.0), (5.5, 1.0),
                        (6.4, 2.2), (5.6, 1.3), (6.6, 1.3), (6.9, 2.1), (6.8, 2.1),
                        (5.7, 1.3), (7.0, 1.4), (6.1, 1.4), (6.1, 1.8), (6.7, 1.7),
                        (6.0, 1.5), (6.5, 1.8), (6.4, 1.5), (6.9, 1.5), (5.6, 1.3),
                        (6.7, 1.4), (5.8, 1.9), (6.3, 1.3), (6.7, 2.1), (6.2, 2.3),
                        (6.3, 2.4), (6.7, 1.8), (6.4, 2.3), (6.2, 1.5), (6.1, 1.4),
                        (7.1, 2.1), (5.7, 1.0), (6.8, 1.4), (6.8, 2.3), (5.1, 1.1),
                        (4.9, 1.7), (5.9, 1.8), (7.4, 1.9), (6.5, 2.0), (6.7, 1.5),
                        (6.5, 2.0), (5.8, 1.0), (6.4, 2.1), (7.6, 2.1), (5.8, 2.4),
                        (7.7, 2.2), (6.3, 1.5), (5.0, 1.0), (6.3, 1.6), (7.7, 2.3),
                        (6.4, 1.9), (6.5, 2.2), (5.7, 1.2), (6.9, 2.3), (5.7, 1.3),
                        (6.1, 1.2), (5.4, 1.5), (5.2, 1.4), (6.7, 2.3), (7.9, 2.0),
                        (5.6, 1.1), (7.2, 1.8), (5.5, 1.3), (7.2, 1.6), (6.3, 2.5),
                        (6.3, 1.8), (6.7, 2.4), (5.0, 1.0), (6.4, 1.8), (6.9, 2.3),
                        (5.5, 1.3), (5.5, 1.1), (5.9, 1.5), (6.0, 1.5), (5.9, 1.8)])

# здесь продолжайте программу
predict = []
for x in x_train:
  with torch.no_grad():
    y = model(x)
    predict.append(y.item())

Task 5

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# здесь объявляйте класс TriagModel
class TriagModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(3, 1, bias=False)

    def forward(self, x):
        x = self.layer1(x)
        return x


np.random.seed(1)
torch.manual_seed(1)

total = 100
x_train = torch.randint(1, 10, (total, 3), dtype=torch.float32)
y_train = x_train.sum(dim=1) / 3

# здесь создавайте модель (model)
# переведите модель в режим обучения
model = TriagModel()
model.eval()

lr = 0.01 # шаг обучения
N = 1000 # число итераций SGD

optimizer = optim.Adam(params=model.parameters(), lr=lr)
loss_func = nn.MSELoss()# здесь создайте функцию потерь с помощью класса nn.MSELoss

for _ in range(N):
    k = np.random.randint(0, total)
    predict = model(x_train[k])
    loss = loss_func(predict, y_train[k])# вычислите значение функции потерь и сохраните результат в переменной loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Task 6

In [33]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# здесь объявляйте класс ClassModel
class ClassModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2,3)
        self.layer2 = nn.Linear(3,1)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        return x


np.random.seed(1)
torch.manual_seed(1)

# обучающая выборка: x_train - входные значения; y_train - целевые значения
x_train = torch.tensor([(5.8, 1.2), (5.6, 1.5), (6.5, 1.5), (6.1, 1.3), (6.4, 1.3), (7.7, 2.0), (6.0, 1.8), (5.6, 1.3), (6.0, 1.6), (5.8, 1.9), (5.7, 2.0), (6.3, 1.5), (6.2, 1.8), (7.7, 2.3), (5.8, 1.2), (6.3, 1.8), (6.0, 1.0), (6.2, 1.3), (5.7, 1.3), (6.3, 1.9), (6.7, 2.5), (5.5, 1.2), (4.9, 1.0), (6.1, 1.4), (6.0, 1.6), (7.2, 2.5), (7.3, 1.8), (6.6, 1.4), (5.6, 2.0), (5.5, 1.0), (6.4, 2.2), (5.6, 1.3), (6.6, 1.3), (6.9, 2.1), (6.8, 2.1), (5.7, 1.3), (7.0, 1.4), (6.1, 1.4), (6.1, 1.8), (6.7, 1.7), (6.0, 1.5), (6.5, 1.8), (6.4, 1.5), (6.9, 1.5), (5.6, 1.3), (6.7, 1.4), (5.8, 1.9), (6.3, 1.3), (6.7, 2.1), (6.2, 2.3), (6.3, 2.4), (6.7, 1.8), (6.4, 2.3), (6.2, 1.5), (6.1, 1.4), (7.1, 2.1), (5.7, 1.0), (6.8, 1.4), (6.8, 2.3), (5.1, 1.1), (4.9, 1.7), (5.9, 1.8), (7.4, 1.9), (6.5, 2.0), (6.7, 1.5), (6.5, 2.0), (5.8, 1.0), (6.4, 2.1), (7.6, 2.1), (5.8, 2.4), (7.7, 2.2), (6.3, 1.5), (5.0, 1.0), (6.3, 1.6), (7.7, 2.3), (6.4, 1.9), (6.5, 2.2), (5.7, 1.2), (6.9, 2.3), (5.7, 1.3), (6.1, 1.2), (5.4, 1.5), (5.2, 1.4), (6.7, 2.3), (7.9, 2.0), (5.6, 1.1), (7.2, 1.8), (5.5, 1.3), (7.2, 1.6), (6.3, 2.5), (6.3, 1.8), (6.7, 2.4), (5.0, 1.0), (6.4, 1.8), (6.9, 2.3), (5.5, 1.3), (5.5, 1.1), (5.9, 1.5), (6.0, 1.5), (5.9, 1.8)])
y_train = torch.FloatTensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1])

model = ClassModel()
model.eval()

total = x_train.size(0) # размер обучающей выборки
N = 1000 # число итераций алгоритма SGD
batch_size = 8

optimizer = optim.Adam(params=model.parameters(), lr=0.01)# задайте оптимизатор Adam с шагом обучения lr=0.01
loss_func = nn.BCEWithLogitsLoss()

for _ in range(N):
    idx = np.random.choice(total, batch_size, False)
    # пропустите через модель k-й образ выборки x_train и вычислите прогноз predict
    predict = model(x_train[idx])
    loss = loss_func(predict.squeeze(1), y_train[idx])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


with torch.no_grad():
    a_new = ([1 if model(x_train[i]) > 0 else 0 for i in range(total)])
    Q = np.array([1 if a_new[i] == y_train[i] else 0 for i in range(total)]).mean()


In [34]:
Q

np.float64(0.93)